<center> Getting started with Tensorflow </center>
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

---
The Computational Graph
---

A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Let's build a simple computational graph. Each node takes zero or more tensors as inputs and produces a tensor as an output. One type of node is a constant. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally. We can create two floating point Tensors node1 and node2 as follows:

In [2]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


Notice that printing the nodes does not output the values 3.0 and 4.0 as you might expect. Instead, they are nodes that, when evaluated, would produce 3.0 and 4.0, respectively. To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the TensorFlow runtime.

The following code creates a Session object and then invokes its run method to run enough of the computational graph to evaluate node1 and node2. By running the computational graph in a session as follows:

In [3]:
sess = tf.Session()
node1, node2 = sess.run([node1, node2])
print(node1, node2)
sess.close()

3.0 4.0


As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.

In [4]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.subtract(a, b)
sess = tf.Session()
c = sess.run(c, feed_dict={a: 5.0, b: 1.4})
print(c)

3.6


In [5]:
sess.close()

In machine learning we will typically want a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. Variables allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [6]:
W = tf.Variable(initial_value=[0.1], dtype=tf.float32)
b = tf.Variable(initial_value=[1], dtype=tf.float32)
X = tf.placeholder(dtype=tf.float32)
y_hat = W * X + b

Constants are initialized when you call tf.constant, and their value can never change. By contrast, variables are not initialized when you call tf.Variable. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
sess.run(y_hat, feed_dict={X: [1,2,3,4]})

array([ 1.10000002,  1.20000005,  1.29999995,  1.39999998], dtype=float32)

In [9]:
y = tf.placeholder(tf.float32)

In [10]:
loss = tf.reduce_mean(tf.squared_difference(y_hat, y))

In [11]:
sess.run(loss, feed_dict={X: [1.0,2.0,3.0,4.0], y:[1.0,2.0,4.0,5.0]})

5.2249999

In [12]:
fixW = tf.assign(W, [0.2])
fixb = tf.assign(b, [1.2])
sess.run([fixW, fixb])

[array([ 0.2], dtype=float32), array([ 1.20000005], dtype=float32)]

In [13]:
print(sess.run([W, b]))

[array([ 0.2], dtype=float32), array([ 1.20000005], dtype=float32)]


---
tf.train API
---

TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. The simplest optimizer is gradient descent. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. In general, computing symbolic derivatives manually is tedious and error-prone. Consequently, TensorFlow can automatically produce derivatives given only a description of the model using the function tf.gradients. For simplicity, optimizers typically do this for you. 

In [14]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

In [15]:
for i in range(1000):
    _, w, c = sess.run([train, W, b], feed_dict={X: [1.0,2.0,3.0,4.0], y: [3.0,5.0,7.0,9.0]})
    print(w, c)

[ 0.45999998] [ 1.28600001]
[ 0.6767] [ 1.35728002]
[ 0.85733098] [ 1.41629946]
[ 1.00791633] [ 1.46510696]
[ 1.13347352] [ 1.505409]
[ 1.23818207] [ 1.53862715]
[ 1.32552338] [ 1.56594551]
[ 1.39839756] [ 1.58835042]
[ 1.45922041] [ 1.60666358]
[ 1.51000416] [ 1.62156928]
[ 1.55242503] [ 1.63363767]
[ 1.58787942] [ 1.64334369]
[ 1.61753035] [ 1.65108287]
[ 1.64234662] [ 1.65718472]
[ 1.66313541] [ 1.66192365]
[ 1.68056893] [ 1.66552842]
[ 1.69520712] [ 1.66818941]
[ 1.70751655] [ 1.67006528]
[ 1.71788585] [ 1.67128813]
[ 1.72663856] [ 1.6719681]
[ 1.73404431] [ 1.67219687]
[ 1.74032784] [ 1.67205071]
[ 1.74567616] [ 1.67159331]
[ 1.75024509] [ 1.67087758]
[ 1.75416446] [ 1.66994774]
[ 1.75754237] [ 1.66884053]
[ 1.76046896] [ 1.66758657]
[ 1.76301932] [ 1.66621137]
[ 1.76525581] [ 1.66473615]
[ 1.76723063] [ 1.66317868]
[ 1.76898706] [ 1.66155362]
[ 1.77056134] [ 1.65987325]
[ 1.7719835] [ 1.65814769]
[ 1.77327859] [ 1.65638554]
[ 1.77446747] [ 1.65459394]
[ 1.77556765] [ 1.65277874]


In [16]:
print(sess.run([W,b]))

[array([ 1.98765266], dtype=float32), array([ 1.03630316], dtype=float32)]


---
tf.contrib.learn
---

tf.contrib.learn is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:

    -running training loops
    -running evaluation loops
    -managing data sets
    -managing feeding
tf.contrib.learn defines many common models.

### Basic Usage

In [19]:
# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,
                                              num_epochs=1000)

# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
estimator.evaluate(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_task_type': None, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8764186d30>, '_save_checkpoints_secs': 600, '_environment': 'local', '_master': '', '_is_chief': True, '_evaluation_master': '', '_tf_random_seed': None}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmprusatr_3/m

{'global_step': 1000, 'loss': 3.2302246e-08}

### A custom model

tf.contrib.learn does not lock you into its predefined models. Suppose we wanted to create a custom model that is not built into TensorFlow. We can still retain the high level abstraction of data set, feeding, training, etc. of tf.contrib.learn. For illustration, we will show how to implement our own equivalent model to LinearRegressor using our knowledge of the lower level TensorFlow API.

To define a custom model that works with tf.contrib.learn, we need to use tf.contrib.learn.Estimator. tf.contrib.learn.LinearRegressor is actually a sub-class of tf.contrib.learn.Estimator. Instead of sub-classing Estimator, we simply provide Estimator a function model_fn that tells tf.contrib.learn how it can evaluate predictions, training steps, and loss. The code is as follows:

In [22]:
def model(features, labels, mode):# Build a linear model and predict values
    # Declare list of features, we only have one real-valued feature
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    # Loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                     tf.assign_add(global_step, 1))
    # ModelFnOps connects subgraphs we built to the
    # appropriate functionality.
    return tf.contrib.learn.ModelFnOps(
        mode=mode, predictions=y,
        loss=loss,
        train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data set
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)

# train
estimator.fit(input_fn=input_fn, steps=1000)
# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_task_type': None, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f87643fae10>, '_save_checkpoints_secs': 600, '_environment': 'local', '_master': '', '_is_chief': True, '_evaluation_master': '', '_tf_random_seed': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp1r1lmplh/model.ckpt.
INFO:tensorflow:loss = 1.09955812321, step = 1
INFO:tensorflow:global_step/sec: 1034.07
INFO:tensorflow:loss = 0.00100986192778, step = 101
INFO:tensorflow:global_step/sec: 898.54
INFO:tensorflow:loss = 1.64997907147e-05, step = 201
INFO:tensorflow:global_step/sec: 1032.14
INFO:tensorflow:loss = 1.02068591281e-05, st